In [23]:
import pandas as pd
df = pd.read_csv('./cars_with_engine_and_isofix.csv')
#df = df[df['variety'].notna()] # remove any NaN values as it blows up serialization
data = df.to_dict('records') # Get only 700 records. More records will make it slower to index
len(data)


16

In [24]:
data

[{'Brand': 'Audi',
  'Model': 'Q7',
  'Engine Size (L)': '3.0',
  'Engine Type': 'Petrol',
  'Number of ISOFIX Points': 6},
 {'Brand': 'Land Rover',
  'Model': 'Discovery',
  'Engine Size (L)': '3.0',
  'Engine Type': 'Diesel',
  'Number of ISOFIX Points': 5},
 {'Brand': 'Kia',
  'Model': 'Sorento',
  'Engine Size (L)': '2.5',
  'Engine Type': 'Petrol',
  'Number of ISOFIX Points': 4},
 {'Brand': 'DS',
  'Model': '7',
  'Engine Size (L)': '1.6',
  'Engine Type': 'Petrol',
  'Number of ISOFIX Points': 3},
 {'Brand': 'Peugeot',
  'Model': '3008',
  'Engine Size (L)': '1.2',
  'Engine Type': 'Petrol',
  'Number of ISOFIX Points': 3},
 {'Brand': 'Volkswagen',
  'Model': 'ID4',
  'Engine Size (L)': 'Electric',
  'Engine Type': 'Electric',
  'Number of ISOFIX Points': 3},
 {'Brand': 'SEAT',
  'Model': 'Tarraco',
  'Engine Size (L)': '1.5',
  'Engine Type': 'Petrol',
  'Number of ISOFIX Points': 3},
 {'Brand': 'Skoda',
  'Model': 'Enyaq',
  'Engine Size (L)': 'Electric',
  'Engine Type': 'Ele

In [25]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [26]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [27]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [28]:
# Create collection to store wines
qdrant.recreate_collection(
    collection_name="car_list",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

/tmp/ipykernel_12084/465066657.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [33]:
# vectorize!
qdrant.upload_points(
    collection_name="car_list",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["Engine Type"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [31]:
user_prompt = "Suggest me an electric car?"

In [34]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name="car_list",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'Brand': 'Hyundai', 'Model': 'Ioniq 5', 'Engine Size (L)': 'Electric', 'Engine Type': 'Electric', 'Number of ISOFIX Points': 2} score: 0.5667376422447183
{'Brand': 'Skoda', 'Model': 'Enyaq', 'Engine Size (L)': 'Electric', 'Engine Type': 'Electric', 'Number of ISOFIX Points': 3} score: 0.5667376422447183
{'Brand': 'Volkswagen', 'Model': 'ID4', 'Engine Size (L)': 'Electric', 'Engine Type': 'Electric', 'Number of ISOFIX Points': 3} score: 0.5667376422447183


In [35]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]
print(search_results)

[{'Brand': 'Hyundai', 'Model': 'Ioniq 5', 'Engine Size (L)': 'Electric', 'Engine Type': 'Electric', 'Number of ISOFIX Points': 2}, {'Brand': 'Skoda', 'Model': 'Enyaq', 'Engine Size (L)': 'Electric', 'Engine Type': 'Electric', 'Number of ISOFIX Points': 3}, {'Brand': 'Volkswagen', 'Model': 'ID4', 'Engine Size (L)': 'Electric', 'Engine Type': 'Electric', 'Number of ISOFIX Points': 3}]


In [39]:
# Now time to connect to the local large language model
from openai import OpenAI
client = OpenAI(
    
    #base_url="http://127.0.0.1:8080/v1", # "http://<Your api-server IP>:port"
    
    api_key = "ollama"
)
completion = client.chat.completions.create(
    model="phi",
    #model="llama3.2",
    
    messages=[
        {"role": "system", "content": "You are chatbot. Your top priority is to recommend a car over different engine types that they are interested in. Keep the answer short and to the point."},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": str(search_results)}
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='\n\n```python\nfrom typing import List, Dict\nimport random\n\nCARS = [{\'Brand\': \'Hyundai\', \'Model\': \'Ioniq 5\', \'Engine Size (L)\': \'Electric\', \'Engine Type\': \'Electric\', \'Number of ISOFIX Points\': 2}, \n {\'Brand\': \'Skoda\', \'Model\': \'Enyaq\', \'Engine Size (L)\': \'Electric\', \'Engine Type\': \'Electric\', \'Number of ISOFIX Points\': 3}, \n {\'Brand\': \'Volkswagen\', \'Model\': \'ID4\', \'Engine Size (L)\': \'Electric\', \'Engine Type\': \'Electric\', \'Number of ISOFIX Points\': 3}]\n\ndef recommend_car(user: Dict[str, str]) -> List[Dict[str, str]]:\n    """This function takes in a dictionary user and returns recommendations for electric cars with 2 to 6 ISOfix points.\n\n    Parameters\n    ----------\n    input: A user\'s preferences as a dictionary {Brand, Color} \n        dict\n\n    Returns\n    -------\n    car_list : List[Dict[str, str]]\n\trecommended car models and their specifications.\n"""\n    if user[\'Engine Type\